# 00_setup — Environment and Provider Smoke Test

This notebook verifies your environment, loads API keys, reads model config, and runs a minimal generation with available providers (OpenRouter, Gemini).

Steps:
- Ensure you have created `.env` from `env.example` and set keys.
- Verify imports and configuration.
- Instantiate clients and run a quick test.



In [1]:
import os
from pathlib import Path

# Ensure working directory is the project root (contains 'src' and 'config')
candidates = [Path.cwd(), Path.cwd().parent, Path.cwd().parent.parent]
project_root = next((c for c in candidates if (c / "src").exists() and (c / "config").exists()), None)
if not project_root:
    raise FileNotFoundError("Could not locate project root; expected 'src' and 'config' near this notebook.")
os.chdir(project_root)
print("Changed CWD to:", os.getcwd())



Changed CWD to: f:\New folder\Research\Project 1


In [2]:
import os
import sys
from pathlib import Path

# Ensure project root (containing 'src') is on sys.path
project_root = None
env_root = os.getenv("PROJECT_ROOT")
if env_root and (Path(env_root) / "src").exists():
    project_root = Path(env_root)
else:
    for candidate in [Path.cwd(), Path.cwd().parent, Path.cwd().parent.parent]:
        if (candidate / "src").exists():
            project_root = candidate
            break

if project_root:
    if str(project_root) not in sys.path:
        sys.path.insert(0, str(project_root))
else:
    raise ModuleNotFoundError(
        "Could not locate project root. Ensure a 'src' directory exists near the notebook, or set PROJECT_ROOT env var."
    )



In [3]:
import os
import json
from pathlib import Path
import yaml

from src.providers.base import GenerationRequest
from src.providers.openrouter_client import OpenRouterClient
# from src.providers.openai_client import OpenAIClient  # Disabled due to budget constraints; using OpenRouter instead
from src.providers.gemini_client import GeminiClient

print("Python version OK. Working dir:", os.getcwd())

# Load model configuration
cfg_path = Path("config/models.yaml")
with cfg_path.open("r", encoding="utf-8") as f:
    models_cfg = yaml.safe_load(f)

# Helpers to pick model ids by provider

def get_model_id(provider: str, default: str) -> str:
    for m in models_cfg.get("models", []):
        if m.get("provider") == provider:
            return m.get("model_id", default)
    return default

clients = {}

# OpenRouter client (requires OPENROUTER_API_KEY)
or_key = os.getenv("OPENROUTER_API_KEY")
if or_key:
    try:
        clients["openrouter-llama"] = OpenRouterClient(model_id=get_model_id("openrouter-llama", "meta-llama/llama-3.1-8b-instruct:free"))
        print("OpenRouter-llama client initialised")
    except Exception as e:
        print("OpenRouter-llama init failed:", e)

    try:
        clients["openrouter-deepseek"] = OpenRouterClient(model_id=get_model_id("openrouter-deepseek", "deepseek/deepseek-chat"))
        print("OpenRouter-deepseek client initialised")
    except Exception as e:
        print("OpenRouter-deepseek init failed:", e)

    try:
        clients["openrouter-kimi"] = OpenRouterClient(model_id=get_model_id("openrouter-kimi", "moonshot/kimi-2"))
        print("OpenRouter-kimi client initialised")
    except Exception as e:
        print("OpenRouter-kimi init failed:", e)

else:
    print("OPENROUTER_API_KEY not set; skipping OpenRouter client")

# OpenAI client (requires OPENAI_API_KEY) — disabled due to budget constraints
# oai_key = os.getenv("OPENAI_API_KEY")
# if oai_key:
#     try:
#         clients["openai"] = OpenAIClient(model_id=get_model_id("openai", "gpt-4o-mini"))
#         print("OpenAI client initialised")
#     except Exception as e:
#         print("OpenAI init failed:", e)
# else:
#     print("OPENAI_API_KEY not set; skipping OpenAI client")

# Gemini client (requires GOOGLE_API_KEY)
gem_key = os.getenv("GOOGLE_API_KEY")
if gem_key:
    try:
        clients["gemini"] = GeminiClient(model_id=get_model_id("gemini", "gemini-flash-latest"))
        print("Gemini client initialised")
    except Exception as e:
        print("Gemini init failed:", e)
else:
    print("GOOGLE_API_KEY not set; skipping Gemini client")

print("Providers available:", list(clients.keys()))



Python version OK. Working dir: f:\New folder\Research\Project 1
OpenRouter-llama client initialised
OpenRouter-deepseek client initialised
OpenRouter-kimi client initialised
Gemini client initialised
Providers available: ['openrouter-llama', 'openrouter-deepseek', 'openrouter-kimi', 'gemini']


f:\New folder\Research\Project 1\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# Smoke test small generation on each client
from src.providers.base import GenerationRequest

for name, client in clients.items():
    try:
        text = client.generate(GenerationRequest(prompt="Hello from 00_setup.", max_tokens=24))
        print(f"[{name}] ->", text[:120].replace("\n", " "))
    except Exception as e:
        print(f"[{name}] generation failed:", e)



[openrouter-llama] -> Hello from the other side! It looks like you're setting up something new. What's the project or initiative you're
[openrouter-deepseek] -> Hello! How can I assist you today?
[openrouter-kimi] -> Hello! It looks like you're starting with the setup step. How can I assist you with the setup process?
[gemini] -> Hello from 00_setup.
